In [1]:
import pandas as pd
import os

from dotenv import load_dotenv
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle


# scikit llm imports
from skllm.config import SKLLMConfig
from skllm.models.gpt.classification.zero_shot import ZeroShotGPTClassifier
from skllm.models.gpt.classification.few_shot import FewShotGPTClassifier

In [2]:
# finds .env file and loads the vars
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY", "Key not found")
openai_org = os.getenv("OPENAI_ORG", "Organization not found")

# Load features

In [3]:
# If this is set to true, we use old data and pickle files that have been created
READ_FROM_PICKLE = True

In [4]:
if READ_FROM_PICKLE:
    malicious_df = pd.read_pickle("data/malicious_features_numeric.pkl")
    benign_df = pd.read_pickle("data/benign_features_numeric.pkl")

# Labeling

In [5]:
# add labels, 0 for benign, 1 for malicious
malicious_df["Labels"] = 1 
benign_df["Labels"] = 0

In [6]:
mirai_payloads = malicious_df[["Payload", "Labels"]]
benign_payloads = benign_df[["Payload", "Labels"]]

In [7]:
payloads = pd.concat([mirai_payloads, benign_payloads], ignore_index=True)

In [8]:
payloads = shuffle(payloads)

In [9]:
# use a subset because this takes loooong!
X_llm = payloads["Payload"].head(100)
y_llm = payloads["Labels"].head(100)
X_test = payloads["Payload"].tail(20)
y_test = payloads["Labels"].tail(20)

In [10]:
SKLLMConfig.set_openai_key(openai_api_key)
SKLLMConfig.set_openai_org(openai_org)

# Models 

## ZeroShotClassifier

In [11]:
clf = ZeroShotGPTClassifier(openai_model="gpt-3.5-turbo")
clf.fit(X_llm, y_llm)
labels = clf.predict(X_llm)

print(f"Accuracy: {accuracy_score(y_llm, labels):.2f}")

100%|██████████| 100/100 [01:22<00:00,  1.21it/s]

Accuracy: 0.66


In [12]:
clf = ZeroShotGPTClassifier(openai_model="gpt-4")
clf.fit(X_llm, y_llm)
labels = clf.predict(X_llm)

print(f"Accuracy: {accuracy_score(y_llm, labels):.2f}")

100%|██████████| 100/100 [01:18<00:00,  1.27it/s]

Accuracy: 0.64


## FewShotClassifier

In [13]:
clf = FewShotGPTClassifier(openai_model="gpt-3.5-turbo")
clf.fit(X_llm, y_llm)
labels = clf.predict(X_llm)

print(f"Accuracy: {accuracy_score(y_llm, labels):.2f}")

100%|██████████| 100/100 [01:22<00:00,  1.21it/s]

Accuracy: 0.90


In [14]:
clf = FewShotGPTClassifier(openai_model="gpt-4")
clf.fit(X_llm, y_llm)
labels = clf.predict(X_llm)

print(f"Accuracy: {accuracy_score(y_llm, labels):.2f}")

100%|██████████| 100/100 [01:23<00:00,  1.19it/s]

Accuracy: 0.90


# Fine Tuning

In [15]:
from skllm.models.gpt.classification.tunable import GPTClassifier

clf = GPTClassifier(
    base_model="gpt-3.5-turbo-0613",
    n_epochs=None,  # int or None. When None, will be determined automatically by OpenAI
    default_label="Random",  # optional
)

clf.fit(X_llm, y_llm)  # y_train is a list of labels
labels = clf.predict(X_test)

Created new file. FILE_ID = file-a65MrjTwy1BrJIbdjwkKOAUv
Waiting for file to be processed ...
Created new tuning job. JOB_ID = ftjob-MrASpQKkbIzZrbct9AIiRlVS
[2024-05-03 11:07:29.755856] Waiting for tuning job to complete. Current status: validating_files
[2024-05-03 11:09:30.020271] Waiting for tuning job to complete. Current status: running
[2024-05-03 11:11:30.373247] Waiting for tuning job to complete. Current status: running
[2024-05-03 11:13:30.794034] Waiting for tuning job to complete. Current status: running
[2024-05-03 11:15:31.036670] Waiting for tuning job to complete. Current status: running
[2024-05-03 11:17:31.413933] Waiting for tuning job to complete. Current status: running
Finished training.


100%|██████████| 20/20 [00:19<00:00,  1.04it/s]


In [16]:
print(f"Accuracy: {accuracy_score(y_test, labels):.2f}")

Accuracy: 0.90
